In [ ]:
pip install pdfplumber


In [145]:
import pdfplumber
import pandas as pd
import os
import numpy as np
import re

pd.set_option('display.max_columns', None) # Mostrar todas as colunas

In [143]:
import pdfplumber
import pandas as pd

# Caminho do arquivo PDF
pdf_path = "C:/Users/Samsung/Documents/GitHub-Datas/Case_Petrobahia/Tabelas de Preços - Gasolina 01-02-25.pdf"

# Dicionário para armazenar DataFrames de cada página
dfs_por_pagina = {}

# Abrir o PDF
with pdfplumber.open(pdf_path) as pdf:
    for i, page in enumerate(pdf.pages, start=1):
        table = page.extract_table()
        if table:
            df = pd.DataFrame(table[1:], columns=table[0])  # Primeira linha como cabeçalho
            dfs_por_pagina[f"pagina_{i}"] = df

# Exemplo: acessar o DataFrame da página 1
print(dfs_por_pagina["pagina_1"].head())


            LOCAL MODALIDADE\nDE VENDA 01.08.2019 16.08.2019 28.08.2019  \
0  Araucária (PR)                  EXA   1.686,70   1.586,70   1.642,80   
1  Araucária (PR)                  LPA   1.688,70   1.588,70   1.644,80   
2  Araucária (PR)                  ETD                                    
3    Barueri (SP)                  EXA   1.788,90   1.688,90   1.745,00   
4    Barueri (SP)                  LPA   1.793,20   1.693,20   1.749,30   

  05.09.2019 19.09.2019 27.09.2019 01.11.2019 19.11.2019 27.11.2019  \
0   1.665,10   1.724,30   1.768,80   1.768,80   1.819,10   1.892,80   
1   1.667,10   1.726,30   1.770,80   1.771,10   1.821,40   1.895,10   
2                                                                     
3   1.767,30   1.826,50   1.871,00   1.871,00   1.921,30   1.995,00   
4   1.771,60   1.830,80   1.875,30   1.875,30   1.925,60   1.999,30   

  01.12.2019 14.01.2020 24.01.2020 31.01.2020 06.02.2020 20.02.2020 29.02.2020  
0   1.892,80   1.859,10   1.832,00   1.77

In [123]:
import numpy as np

# Função para converter números no formato brasileiro para float
def converter_para_float(valor):
    if isinstance(valor, str):  # Verifica se é string
        valor = valor.strip()  # Remove espaços em branco extras
        if valor == "":  # Se for string vazia, retorna NaN
            return np.nan
        try:
            return float(valor.replace(".", "").replace(",", "."))  # Converte para float
        except ValueError:  # Se não puder converter, retorna NaN
            return np.nan
    return valor  # Se já for número, retorna o valor original

# Lista das colunas que queremos manter como texto
colunas_texto = ["LOCAL", "MODALIDADE DE VENDA"]

# Loop para percorrer de "pagina_1" a "pagina_9"
for i in range(1, 10):
    pagina = f"pagina_{i}"
    
    # Renomear a coluna 'MODALIDADE\nDE VENDA' para 'MODALIDADE DE VENDA'
    dfs_por_pagina[pagina] = dfs_por_pagina[pagina].rename(columns={'MODALIDADE\nDE VENDA': 'MODALIDADE DE VENDA'})
    
    # Aplicar a conversão para todas as colunas, exceto as de texto
    dfs_por_pagina[pagina] = dfs_por_pagina[pagina].apply(
        lambda col: col.apply(converter_para_float) if col.name not in colunas_texto else col
    )
    
    # Verificar os tipos das colunas
    print(f"Informações da {pagina}:")
    dfs_por_pagina[pagina].info()

# Concatenar todos os DataFrames, unindo-os lateralmente
df_geral = pd.concat([dfs_por_pagina[f"pagina_{i}"] for i in range(1, 10)], axis=1)

# Verificar o DataFrame final
df_geral.head()


Informações da pagina_1:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   LOCAL                101 non-null    object 
 1   MODALIDADE DE VENDA  101 non-null    object 
 2   01.08.2019           71 non-null     float64
 3   16.08.2019           71 non-null     float64
 4   28.08.2019           71 non-null     float64
 5   05.09.2019           71 non-null     float64
 6   19.09.2019           71 non-null     float64
 7   27.09.2019           71 non-null     float64
 8   01.11.2019           70 non-null     float64
 9   19.11.2019           70 non-null     float64
 10  27.11.2019           70 non-null     float64
 11  01.12.2019           70 non-null     float64
 12  14.01.2020           70 non-null     float64
 13  24.01.2020           70 non-null     float64
 14  31.01.2020           70 non-null     float64
 15  06.02.2020     

,LOCAL,MODALIDADE DE VENDA,01.08.2019,16.08.2019,28.08.2019,05.09.2019,19.09.2019,27.09.2019,01.11.2019,19.11.2019,27.11.2019,01.12.2019,14.01.2020,24.01.2020,31.01.2020,06.02.2020,20.02.2020,29.02.2020,LOCAL,MODALIDADE DE VENDA,01.03.2020,13.03.2020,19.03.2020,25.03.2020,28.03.2020,01.04.2020,15.04.2020,17.04.2020,21.04.2020,29.04.2020,07.05.2020,14.05.2020,21.05.2020,27.05.2020,01.06.2020,09.06.2020,LOCAL,MODALIDADE DE VENDA,19.06.2020,01.07.2020,02.07.2020,08.07.2020,17.07.2020,31.07.2020,01.08.2020,13.08.2020,21.08.2020,26.08.2020,03.09.2020,09.09.2020,11.09.2020,23.09.2020,30.09.2020,01.10.2020,LOCAL,MODALIDADE DE VENDA,10.10.2020,16.10.2020,27.10.2020,31.10.2020,01.11.2020,12.11.2020,26.11.2020,03.12.2020,16.12.2020,29.12.2020,18.01.2021,19.01.2021,27.01.2021,01.02.2021,09.02.2021,19.02.2021,LOCAL,MODALIDADE DE VENDA,01.03.2021,02.03.2021,09.03.2021,20.03.2021,25.03.2021,01.04.2021,16.04.2021,01.05.2021,01.06.2021,12.06.2021,01.07.2021,06.07.2021,01.08.2021,12.08.2021,01.09.2021,01.10.2021,LOCAL,MODALIDADE DE VENDA,09.10.2021,26.10.2021,01.11.2021,01.12.2021,15.12.2021,01.01.2022,12.01.2022,01.02.2022,01.03.2022,11.03.2022,01.04.2022,01.05.2022,01.06.2022,18.06.2022,24.06.2022,20.07.2022,LOCAL,MODALIDADE DE VENDA,29.07.2022,01.08.2022,16.08.2022,01.09.2022,02.09.2022,01.10.2022,01.11.2022,07.12.2022,01.01.2023,25.01.2023,01.02.2023,01.03.2023,01.04.2023,01.05.2023,17.05.2023,01.06.2023,LOCAL,MODALIDADE DE VENDA,08.06.2023,16.06.2023,01.07.2023,01.08.2023,16.08.2023,01.09.2023,01.10.2023,21.10.2023,01.11.2023,01.12.2023,01.01.2024,01.02.2024,01.03.2024,01.04.2024,01.05.2024,01.06.2024,LOCAL,MODALIDADE DE VENDA,01.07.2024,09.07.2024,01.09.2024,01.10.2024,01.11.2024,01.12.2024,01.01.2025,01.02.2025
0,Araucária (PR),EXA,1686.7,1586.7,1642.8,1665.1,1724.3,1768.8,1768.8,1819.1,1892.8,1892.8,1859.1,1832.0,1776.8,1701.2,1752.4,1682.3,Araucária (PR),EXA,1682.3,1522.3,1340.3,1139.6,1083.0,1083.0,997.0,997.0,917.6,917.6,1027.3,1129.7,1264.7,1327.5,1327.5,1460.2,Araucária (PR),EXA,1533.4,1533.4,1579.1,1658.5,1724.9,1655.7,1655.7,1721.9,1825.1,1825.1,1771.1,1682.7,1599.3,1663.9,1747.0,1747.0,Araucária (PR),EXA,1817.0,1744.4,1657.3,1574.4,1574.4,1669.1,1735.9,1701.1,1752.9,1839.6,1839.6,1986.5,2087.1,2087.1,2255.4,2481.6,Araucária (PR),EXA,2481.6,2605.6,2839.8,2697.8,2589.8,2589.8,2639.5,2586.6,2586.6,2535.2,2535.2,2692.3,2692.3,2785.8,2785.8,2785.8,Araucária (PR),EXA,2978.9,3187.7,3187.7,3187.7,3091.9,3091.9,3246.5,3246.5,3246.5,3855.8,3855.8,3855.8,3855.8,4055.8,4055.8,3855.0,Araucária (PR),EXA,3705.0,3705.0,3526.4,3526.4,3276.4,3276.4,3276.4,3076.4,3076.4,3306.4,3306.4,3176.4,3176.4,3176.4,2776.4,2776.4,Araucária (PR),EXA,2776.4,2646.4,2506.4,2506.4,2916.4,2916.4,2916.4,2790.9,2790.9,2790.9,2790.9,2790.9,2790.9,2790.9,2790.9,2790.9,Araucária (PR),EXA,2790.9,2990.9,2990.9,2990.9,2990.9,2990.9,2990.9,2990.9
1,Araucária (PR),LPA,1688.7,1588.7,1644.8,1667.1,1726.3,1770.8,1771.1,1821.4,1895.1,1895.1,1861.4,1834.3,1779.1,1703.5,1754.7,1684.6,Araucária (PR),LPA,1684.6,1524.6,1342.6,1141.9,1085.3,1085.3,999.3,999.3,919.9,919.9,1029.6,1132.0,1267.0,1329.8,1329.8,1462.5,Araucária (PR),LPA,1535.7,1535.7,1581.4,1660.8,1727.2,1658.0,1658.0,1724.2,1827.4,1827.4,1773.4,1685.0,1601.6,1666.2,1749.3,1749.3,Araucária (PR),LPA,1819.3,1746.7,1659.6,1576.7,1576.7,1671.4,1738.2,1703.4,1755.2,1841.9,1841.9,1988.8,2089.4,2089.4,2257.7,2483.9,Araucária (PR),LPA,2483.9,2607.9,2842.1,2700.1,2592.1,2592.1,2641.8,2588.9,2588.9,2537.5,2537.5,2694.6,2694.6,2788.1,2788.1,2788.1,Araucária (PR),LPA,2981.2,3190.0,3190.0,3190.0,3094.2,3094.2,3248.8,3248.8,3248.8,3858.1,3858.1,3858.1,3858.1,4058.1,4058.1,3857.3,Araucária (PR),LPA,3707.3,3707.3,3528.7,3528.7,3278.7,3278.7,3278.7,3078.7,3078.7,3308.7,3308.7,3178.7,3178.7,3178.9,2778.9,2778.9,Araucária (PR),LPA,2778.9,2648.9,2508.9,2508.9,2918.9,2918.9,2918.9,2793.4,2793.4,2793.4,2793.4,2793.4,2793.4,2793.6,2793.6,2793.6,Araucária (PR),LPA,2793.6,2993.6,2993.6,2993.6,2993.6,2993.6,2993.6,2993.6
2,Araucária (PR),ETD,NaN,Na

In [124]:
# Remover colunas duplicadas, mantendo apenas a primeira
df_geral = df_geral.loc[:, ~df_geral.columns.duplicated()]
df_geral.head()

,LOCAL,MODALIDADE DE VENDA,01.08.2019,16.08.2019,28.08.2019,05.09.2019,19.09.2019,27.09.2019,01.11.2019,19.11.2019,27.11.2019,01.12.2019,14.01.2020,24.01.2020,31.01.2020,06.02.2020,20.02.2020,29.02.2020,01.03.2020,13.03.2020,19.03.2020,25.03.2020,28.03.2020,01.04.2020,15.04.2020,17.04.2020,21.04.2020,29.04.2020,07.05.2020,14.05.2020,21.05.2020,27.05.2020,01.06.2020,09.06.2020,19.06.2020,01.07.2020,02.07.2020,08.07.2020,17.07.2020,31.07.2020,01.08.2020,13.08.2020,21.08.2020,26.08.2020,03.09.2020,09.09.2020,11.09.2020,23.09.2020,30.09.2020,01.10.2020,10.10.2020,16.10.2020,27.10.2020,31.10.2020,01.11.2020,12.11.2020,26.11.2020,03.12.2020,16.12.2020,29.12.2020,18.01.2021,19.01.2021,27.01.2021,01.02.2021,09.02.2021,19.02.2021,01.03.2021,02.03.2021,09.03.2021,20.03.2021,25.03.2021,01.04.2021,16.04.2021,01.05.2021,01.06.2021,12.06.2021,01.07.2021,06.07.2021,01.08.2021,12.08.2021,01.09.2021,01.10.2021,09.10.2021,26.10.2021,01.11.2021,01.12.2021,15.12.2021,01.01.2022,12.01.2022,01.02.2022,01.03.2022,11.03.2022,01.04.2022,01.05.2022,01.06.2022,18.06.2022,24.06.2022,20.07.2022,29.07.2022,01.08.2022,16.08.2022,01.09.2022,02.09.2022,01.10.2022,01.11.2022,07.12.2022,01.01.2023,25.01.2023,01.02.2023,01.03.2023,01.04.2023,01.05.2023,17.05.2023,01.06.2023,08.06.2023,16.06.2023,01.07.2023,01.08.2023,16.08.2023,01.09.2023,01.10.2023,21.10.2023,01.11.2023,01.12.2023,01.01.2024,01.02.2024,01.03.2024,01.04.2024,01.05.2024,01.06.2024,01.07.2024,09.07.2024,01.09.2024,01.10.2024,01.11.2024,01.12.2024,01.01.2025,01.02.2025
0,Araucária (PR),EXA,1686.7,1586.7,1642.8,1665.1,1724.3,1768.8,1768.8,1819.1,1892.8,1892.8,1859.1,1832.0,1776.8,1701.2,1752.4,1682.3,1682.3,1522.3,1340.3,1139.6,1083.0,1083.0,997.0,997.0,917.6,917.6,1027.3,1129.7,1264.7,1327.5,1327.5,1460.2,1533.4,1533.4,1579.1,1658.5,1724.9,1655.7,1655.7,1721.9,1825.1,1825.1,1771.1,1682.7,1599.3,1663.9,1747.0,1747.0,1817.0,1744.4,1657.3,1574.4,1574.4,1669.1,1735.9,1701.1,1752.9,1839.6,1839.6,1986.5,2087.1,2087.1,2255.4,2481.6,2481.6,2605.6,2839.8,2697.8,2589.8,2589.8,2639.5,2586.6,2586.6,2535.2,2535.2,2692.3,2692.3,2785.8,2785.8,2785.8,2978.9,3187.7,3187.7,3187.7,3091.9,3091.9,3246.5,3246.5,3246.5,3855.8,3855.8,3855.8,3855.8,4055.8,4055.8,3855.0,3705.0,3705.0,3526.4,3526.4,3276.4,3276.4,3276.4,3076.4,3076.4,3306.4,3306.4,3176.4,3176.4,3176.4,2776.4,2776.4,2776.4,2646.4,2506.4,2506.4,2916.4,2916.4,2916.4,2790.9,2790.9,2790.9,2790.9,2790.9,2790.9,2790.9,2790.9,2790.9,2790.9,2990.9,2990.9,2990.9,2990.9,2990.9,2990.9,2990.9
1,Araucária (PR),LPA,1688.7,1588.7,1644.8,1667.1,1726.3,1770.8,1771.1,1821.4,1895.1,1895.1,1861.4,1834.3,1779.1,1703.5,1754.7,1684.6,1684.6,1524.6,1342.6,1141.9,1085.3,1085.3,999.3,999.3,919.9,919.9,1029.6,1132.0,1267.0,1329.8,1329.8,1462.5,1535.7,1535.7,1581.4,1660.8,1727.2,1658.0,1658.0,1724.2,1827.4,1827.4,1773.4,1685.0,1601.6,1666.2,1749.3,1749.3,1819.3,1746.7,1659.6,1576.7,1576.7,1671.4,1738.2,1703.4,1755.2,1841.9,1841.9,1988.8,2089.4,2089.4,2257.7,2483.9,2483.9,2607.9,2842.1,2700.1,2592.1,2592.1,2641.8,2588.9,2588.9,2537.5,2537.5,2694.6,2694.6,2788.1,2788.1,2788.1,2981.2,3190.0,3190.0,3190.0,3094.2,3094.2,3248.8,3248.8,3248.8,3858.1,3858.1,3858.1,3858.1,4058.1,4058.1,3857.3,3707.3,3707.3,3528.7,3528.7,3278.7,3278.7,3278.7,3078.7,3078.7,3308.7,3308.7,3178.7,3178.7,3178.9,2778.9,2778.9,2778.9,2648.9,2508.9,2508.9,2918.9,2918.9,2918.9,2793.4,2793.4,2793.4,2793.4,2793.4,2793.4,2793.6,2793.6,2793.6,2793.6,2993.6,2993.6,2993.6,2993.6,2993.6,2993.6,2993.6
2,Araucária (PR),ETD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4070.8,3870.0,3720.0,3720.0,3541.4,3541.4,3291.4,3291.4,3291.4,3091.4,3091.4,3321.4,3321.4,3191.4,3191.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [125]:
df_geral.to_excel('C:/Users/Samsung/Documents/GitHub-Datas/Case_Petrobahia/Tabela_de_Preco_Geral.xlsx', index=False)

#### Tratamento pagina 10.

In [102]:
import pdfplumber
import pandas as pd

# Caminho do arquivo PDF
pdf_path_10 = "C:/Users/Samsung/Documents/GitHub-Datas/Case_Petrobahia/Tabelas de Preços - Gasolina 01-02-25.pdf"

# Lista para armazenar DataFrames de cada tabela encontrada na página 10
tabelas_pag10 = []

# Abrindo o PDF e extraindo apenas a página 10 (índice 9, pois começa em 0)
with pdfplumber.open(pdf_path_10) as pdf:
    pagina10 = pdf.pages[9]  # Página 10 tem índice 9
    tabelas = pagina10.extract_tables()

    # Converter cada tabela extraída em um DataFrame sem ajustar cabeçalho
    for tabela in tabelas:
        df_temp = pd.DataFrame(tabela)  # Não definir cabeçalho
        tabelas_pag10.append(df_temp)

# Concatenar todas as tabelas extraídas em um único DataFrame
df_pag10 = pd.concat(tabelas_pag10, ignore_index=True)

# Exibir o DataFrame resultante
df_pag10


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,LOCAL,MODALIDADE\nDE VENDA,01.08.2019,16.08.2019,28.08.2019,05.09.2019,19.09.2019,27.09.2019,01.11.2019,19.11.2019,27.11.2019,14.01.2020,24.01.2020,31.01.2020,06.02.2020,20.02.2020,29.02.2020
1,Duque de Caxias (RJ),EXA,"2.104,36","2.004,36","2.060,46","2.082,76","2.141,96","2.186,46","2.186,46","2.236,76","2.310,46","2.301,06","2.273,96","2.218,76","2.143,16","2.194,36","2.124,26"
2,Duque de Caxias (RJ),LPA,"2.106,36","2.006,36","2.062,46","2.084,76","2.143,96","2.188,46","2.188,76","2.239,06","2.312,76","2.303,36","2.276,26","2.221,06","2.145,46","2.196,66","2.126,56"
3,LOCAL,MODALIDADE\nDE VENDA,13.03.2020,19.03.2020,25.03.2020,28.03.2020,15.04.2020,21.04.2020,07.05.2020,14.05.2020,21.05.2020,27.05.2020,09.06.2020,19.06.2020,02.07.2020,08.07.2020,17.07.2020
4,Duque de Caxias (RJ),EXA,"1.964,26","1.782,26","1.581,56","1.524,96","1.438,96","1.359,56","1.469,26","1.571,66","1.706,66","1.769,46","1.902,16","1.975,36","2.021,06","2.100,46","2.166,86"
5,Duque de Caxias (RJ),LPA,"1.966,56","1.784,56","1.583,86","1.527,26","1.441,26","1.361,86","1.471,56","1.573,96","1.708,96","1.771,76","1.904,46","1.977,66","2.023,36","2.102,76","2.169,16"
6,LOCAL,MODALIDADE\nDE VENDA,31.07.2020,13.08.2020,21.08.2020,03.09.2020,09.09.2020,11.09.2020,23.09.2020,30.09.2020,10.10.2020,16.10.2020,27.10.2020,31.10.2020,12.11.2020,26.11.2020,03.12.2020
7,Duque de Caxias (RJ),EXA,"2.097,66","2.163,86","2.267,06","2.213,06","2.124,66","2.041,26","2.105,86","2.188,96","2.258,96","2.186,36","2.099,26","2.016,36","2.111,06","2.177,86","2.143,06"
8,Duque de Caxias (RJ),LPA,"2.099,96","2.166,16","2.269,36","2.215,36","2.126,96","2.043,56","2.108,16","2.191,26","2.261,26","2.188,66","2.101,56","2.018,66","2.113,36","2.180,16","2.145,36"
9,LOCAL,MODALIDADE\nDE VENDA,16.12.2020,29.12.2020,19.01.2021,27.01.2021,09.02.2021,19.02.2021,02.03.2021,09.03.2021,20.03.2021,25.03.2021,16.04.2021,01.05.2021,12.06.2021,06.07.2021,12.08.2021


In [103]:
def processar_e_unir(df, passo=3):
    df_final = None  # Inicializa a variável para armazenar a união dos DataFrames

    for i in range(0, len(df), passo):
        df_temp = df.iloc[i:i+passo].copy().reset_index(drop=True)  # Seleciona o bloco
        df_temp.columns = df_temp.iloc[0]  # Define a primeira linha como cabeçalho
        df_temp = df_temp[1:].reset_index(drop=True)  # Remove a primeira linha
        df_temp = df_temp.rename(columns={'MODALIDADE\nDE VENDA': 'MODALIDADE DE VENDA'})  # Renomeia coluna

        # Junta os DataFrames iterativamente
        if df_final is None:
            df_final = df_temp  # Primeiro bloco se torna o inicial
        else:
            df_final = df_final.merge(df_temp, on=["MODALIDADE DE VENDA", "LOCAL"], how="outer")

    return df_final  # Retorna o DataFrame final unido

# Aplicando a função no df_pag10
df_pag10_union = processar_e_unir(df_pag10)
df_pag10_union.head()


,LOCAL,MODALIDADE DE VENDA,01.08.2019,16.08.2019,28.08.2019,05.09.2019,19.09.2019,27.09.2019,01.11.2019,19.11.2019,27.11.2019,14.01.2020,24.01.2020,31.01.2020,06.02.2020,20.02.2020,29.02.2020,13.03.2020,19.03.2020,25.03.2020,28.03.2020,15.04.2020,21.04.2020,07.05.2020,14.05.2020,21.05.2020,27.05.2020,09.06.2020,19.06.2020,02.07.2020,08.07.2020,17.07.2020,31.07.2020,13.08.2020,21.08.2020,03.09.2020,09.09.2020,11.09.2020,23.09.2020,30.09.2020,10.10.2020,16.10.2020,27.10.2020,31.10.2020,12.11.2020,26.11.2020,03.12.2020,16.12.2020,29.12.2020,19.01.2021,27.01.2021,09.02.2021,19.02.2021,02.03.2021,09.03.2021,20.03.2021,25.03.2021,16.04.2021,01.05.2021,12.06.2021,06.07.2021,12.08.2021,09.10.2021,26.10.2021,15.12.2021,12.01.2022,11.03.2022,18.06.2022,20.07.2022,29.07.2022,16.08.2022,02.09.2022,07.12.2022,25.01.2023,01.03.2023,01.04.2023,01.05.2023,17.05.2023,16.06.2023,01.07.2023,16.08.2023,21.10.2023,01.04.2024,09.07.2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Duque de Caxias (RJ),EXA,"2.104,36","2.004,36","2.060,46","2.082,76","2.141,96","2.186,46","2.186,46","2.236,76","2.310,46","2.301,06","2.273,96","2.218,76","2.143,16","2.194,36","2.124,26","1.964,26","1.782,26","1.581,56","1.524,96","1.438,96","1.359,56","1.469,26","1.571,66","1.706,66","1.769,46","1.902,16","1.975,36","2.021,06","2.100,46","2.166,86","2.097,66","2.163,86","2.267,06","2.213,06","2.124,66","2.041,26","2.105,86","2.188,96","2.258,96","2.186,36","2.099,26","2.016,36","2.111,06","2.177,86","2.143,06","2.194,86","2.281,56","2.428,46","2.529,06","2.697,36","2.923,56","3.047,56","3.281,76","3.139,76","3.031,76","3.081,46","3.028,56","2.977,16","3.134,26","3.227,76","3.420,86","3.629,66","3.533,86","3.688,46","4.297,76","4.497,76","4.296,96","4.146,96","3.968,36","3.718,36","3.518,36","3.748,36","3.618,36",,,,,,,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Duque de Caxias (RJ),LPA,"2.106,36","2.006,36","2.062,46","2.084,76","2.143,96","2.188,46","2.188,76","2.239,06","2.312,76","2.303,36","2.276,26","2.221,06","2.145,46","2.196,66","2.126,56","1.966,56","1.784,56","1.583,86","1.527,26","1.441,26","1.361,86","1.471,56","1.573,96","1.708,96","1.771,76","1.904,46","1.977,66","2.023,36","2.102,76","2.169,16","2.099,96","2.166,16","2.269,36","2.215,36","2.126,96","2.043,56","2.108,16","2.191,26","2.261,26","2.188,66","2.101,56","2.018,66","2.113,36","2.180,16","2.145,36","2.197,16","2.283,86","2.430,76","2.531,36","2.699,66","2.925,86","3.049,86","3.284,06","3.142,06","3.034,06","3.083,76","3.030,86","2.979,46","3.136,56","3.230,06","3.423,16","3.631,96","3.536,16","3.690,76","4.300,06","4.500,06","4.299,26","4.149,26","3.970,66","3.720,66","3.520,66","3.750,66","3.620,66","3.746,70","3.746,90","3.346,90","3.216,90","3.076,90","3.486,90","3.361,40","3.361,60","3.561,60",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
df_pag10_union = df_pag10_union.drop(columns=[col for col in df_pag10_union.columns if pd.isna(col)])
df_pag10_union.head()


,LOCAL,MODALIDADE DE VENDA,01.08.2019,16.08.2019,28.08.2019,05.09.2019,19.09.2019,27.09.2019,01.11.2019,19.11.2019,27.11.2019,14.01.2020,24.01.2020,31.01.2020,06.02.2020,20.02.2020,29.02.2020,13.03.2020,19.03.2020,25.03.2020,28.03.2020,15.04.2020,21.04.2020,07.05.2020,14.05.2020,21.05.2020,27.05.2020,09.06.2020,19.06.2020,02.07.2020,08.07.2020,17.07.2020,31.07.2020,13.08.2020,21.08.2020,03.09.2020,09.09.2020,11.09.2020,23.09.2020,30.09.2020,10.10.2020,16.10.2020,27.10.2020,31.10.2020,12.11.2020,26.11.2020,03.12.2020,16.12.2020,29.12.2020,19.01.2021,27.01.2021,09.02.2021,19.02.2021,02.03.2021,09.03.2021,20.03.2021,25.03.2021,16.04.2021,01.05.2021,12.06.2021,06.07.2021,12.08.2021,09.10.2021,26.10.2021,15.12.2021,12.01.2022,11.03.2022,18.06.2022,20.07.2022,29.07.2022,16.08.2022,02.09.2022,07.12.2022,25.01.2023,01.03.2023,01.04.2023,01.05.2023,17.05.2023,16.06.2023,01.07.2023,16.08.2023,21.10.2023,01.04.2024,09.07.2024
0,Duque de Caxias (RJ),EXA,"2.104,36","2.004,36","2.060,46","2.082,76","2.141,96","2.186,46","2.186,46","2.236,76","2.310,46","2.301,06","2.273,96","2.218,76","2.143,16","2.194,36","2.124,26","1.964,26","1.782,26","1.581,56","1.524,96","1.438,96","1.359,56","1.469,26","1.571,66","1.706,66","1.769,46","1.902,16","1.975,36","2.021,06","2.100,46","2.166,86","2.097,66","2.163,86","2.267,06","2.213,06","2.124,66","2.041,26","2.105,86","2.188,96","2.258,96","2.186,36","2.099,26","2.016,36","2.111,06","2.177,86","2.143,06","2.194,86","2.281,56","2.428,46","2.529,06","2.697,36","2.923,56","3.047,56","3.281,76","3.139,76","3.031,76","3.081,46","3.028,56","2.977,16","3.134,26","3.227,76","3.420,86","3.629,66","3.533,86","3.688,46","4.297,76","4.497,76","4.296,96","4.146,96","3.968,36","3.718,36","3.518,36","3.748,36","3.618,36",,,,,,,,,
1,Duque de Caxias (RJ),LPA,"2.106,36","2.006,36","2.062,46","2.084,76","2.143,96","2.188,46","2.188,76","2.239,06","2.312,76","2.303,36","2.276,26","2.221,06","2.145,46","2.196,66","2.126,56","1.966,56","1.784,56","1.583,86","1.527,26","1.441,26","1.361,86","1.471,56","1.573,96","1.708,96","1.771,76","1.904,46","1.977,66","2.023,36","2.102,76","2.169,16","2.099,96","2.166,16","2.269,36","2.215,36","2.126,96","2.043,56","2.108,16","2.191,26","2.261,26","2.188,66","2.101,56","2.018,66","2.113,36","2.180,16","2.145,36","2.197,16","2.283,86","2.430,76","2.531,36","2.699,66","2.925,86","3.049,86","3.284,06","3.142,06","3.034,06","3.083,76","3.030,86","2.979,46","3.136,56","3.230,06","3.423,16","3.631,96","3.536,16","3.690,76","4.300,06","4.500,06","4.299,26","4.149,26","3.970,66","3.720,66","3.520,66","3.750,66","3.620,66","3.746,70","3.746,90","3.346,90","3.216,90","3.076,90","3.486,90","3.361,40","3.361,60","3.561,60"


In [ ]:
import numpy as np
import pandas as pd

# Função para converter números no formato brasileiro para float
def converter_para_float(valor):
    if isinstance(valor, str):  # Verifica se é string
        valor = valor.strip()  # Remove espaços em branco extras
        if valor == "":  # Se for string vazia, retorna NaN
            return np.nan
        try:
            return float(valor.replace(".", "").replace(",", "."))  # Converte para float
        except ValueError:  # Se não puder converter, retorna NaN
            return np.nan
    return valor  # Se já for número, retorna o valor original

# Lista das colunas que queremos manter como texto
colunas_texto = ["LOCAL", "MODALIDADE DE VENDA"]

# Aplicar a conversão a todas as colunas, exceto as que queremos manter como texto
df_pag10_union = df_pag10_union.apply(
    lambda col: col.apply(converter_para_float) if col.name not in colunas_texto else col
)

# Verificar os tipos das colunas
df_pag10_union.info()


In [137]:
df_pag10_union.to_excel('C:/Users/Samsung/Documents/GitHub-Datas/Case_Petrobahia/Tabela_de_Preco_Pagina_10.xlsx', index=False)

#### Tratamento página 11.

In [149]:
import re

# Função para corrigir as palavras unidas
def corrigir_palavras_unidas(texto):
    # Substituir todas as quebras de linha (\n) por espaço
    texto = texto.replace('\n', ' ')
    
    # Expressão regular para identificar palavras coladas, mesmo quando não há mudança de maiúscula/minúscula
    texto_corrigido = re.sub(r'([a-z])([A-Z])', r'\1 \2', texto)  # Corrige a união de minúscula e maiúscula
    texto_corrigido = re.sub(r'([a-zA-Z])([0-9])', r'\1 \2', texto_corrigido)  # Corrige a união de palavras com números
    texto_corrigido = re.sub(r'([0-9])([a-zA-Z])', r'\1 \2', texto_corrigido)  # Corrige a união de números com palavras
    return texto_corrigido

# Aplicar a função de correção
dfs_por_pagina["pagina_11"]['Legenda'] = dfs_por_pagina["pagina_11"]['Legenda'].apply(corrigir_palavras_unidas)
dfs_por_pagina["pagina_11"]['Descrição'] = dfs_por_pagina["pagina_11"]['Descrição'].apply(corrigir_palavras_unidas)

# Verificar o resultado
dfs_por_pagina["pagina_11"].head()


,Modalidade de Venda,Legenda,Descrição
0,EXA,Ex-Ponto “A”,"Entrega por duto a serviço da COMPRADORA, inte..."
1,LPA,Livre para o Armazém,"Entrega por duto, ou trecho de duto, a serviço..."
2,LCT,Livre no Compartimento de Carga do Veículo Rec...,Entrega diretamente no Veiculo Recebedor estac...
3,LPC,Livre para Carregamento,Entrega diretamente na Estação de Carregamento...
4,ETM,Entrega no Tanque de Destino Marítimo,Entrega no terminal de destino a partir de nav...


In [152]:
pip install ftfy

     -------------------------------------- 43.0/43.0 kB 697.3 kB/s eta 0:00:00
  Attempting uninstall: wcwidth
    Found existing installation: wcwidth 0.2.5
    Uninstalling wcwidth-0.2.5:
      Successfully uninstalled wcwidth-0.2.5
Note: you may need to restart the kernel to use updated packages.


In [153]:
import ftfy

# Função para corrigir palavras unidas usando ftfy
def corrigir_palavras_unidas(texto):
    # Usar o ftfy para corrigir o texto
    return ftfy.fix_text(texto)

# Aplicar a função de correção com ftfy
dfs_por_pagina["pagina_11"]['Legenda'] = dfs_por_pagina["pagina_11"]['Legenda'].apply(corrigir_palavras_unidas)
dfs_por_pagina["pagina_11"]['Descrição'] = dfs_por_pagina["pagina_11"]['Descrição'].apply(corrigir_palavras_unidas)

# Verificar o resultado
dfs_por_pagina["pagina_11"].head()


,Modalidade de Venda,Legenda,Descrição
0,EXA,"Ex-Ponto ""A""","Entrega por duto a serviço da COMPRADORA, inte..."
1,LPA,Livre para o Armazém,"Entrega por duto, ou trecho de duto, a serviço..."
2,LCT,Livre no Compartimento de Carga do Veículo Rec...,Entrega diretamente no Veiculo Recebedor estac...
3,LPC,Livre para Carregamento,Entrega diretamente na Estação de Carregamento...
4,ETM,Entrega no Tanque de Destino Marítimo,Entrega no terminal de destino a partir de nav...


In [155]:
print(dfs_por_pagina["pagina_11"]['Descrição'].dtype)


object


In [156]:
dfs_por_pagina["pagina_11"]['Descrição'] = dfs_por_pagina["pagina_11"]['Descrição'].astype(str)

In [159]:
# Exemplo de texto na célula
print(dfs_por_pagina["pagina_11"]['Descrição'].iloc[6])

Entrega diretamente na Estação de Descarregamento a serviço da COMPRADORA a partir da descargado Veículode Entrega,oudiretamentenotanquerecebedor,quandoaentregaseder por duto a serviço da PETROBRAS.


In [162]:
import re

# Função para corrigir palavras unidas
def corrigir_palavras_unidas(texto):
    # Garantir que é uma string
    texto = str(texto)
    
    # Substituir quebras de linha por espaço
    texto = texto.replace('\n', ' ')
    
    # Corrigir palavras unidas por maiúsculas
    texto_corrigido = re.sub(r'([a-z])([A-Z])', r'\1 \2', texto)
    
    # Corrigir outras palavras coladas (letras e números)
    texto_corrigido = re.sub(r'([a-zA-Z])([0-9])', r'\1 \2', texto_corrigido)
    texto_corrigido = re.sub(r'([0-9])([a-zA-Z])', r'\1 \2', texto_corrigido)
    
    return texto_corrigido

# Testar em uma amostra específica
dfs_por_pagina["pagina_11"]['Descrição'] = dfs_por_pagina["pagina_11"]['Descrição'].apply(corrigir_palavras_unidas)

# Verificar o resultado
print(dfs_por_pagina["pagina_11"]['Descrição'].iloc[6])

Entrega diretamente na Estação de Descarregamento a serviço da COMPRADORA a partir da descargado Veículode Entrega,oudiretamentenotanquerecebedor,quandoaentregaseder por duto a serviço da PETROBRAS.


In [161]:
# Verificar e corrigir a codificação se necessário
dfs_por_pagina["pagina_11"]['Descrição'] = dfs_por_pagina["pagina_11"]['Descrição'].apply(lambda x: x.encode('utf-8').decode('utf-8'))


In [154]:
dfs_por_pagina["pagina_11"].to_excel('C:/Users/Samsung/Documents/GitHub-Datas/Case_Petrobahia/Tabela_de_Preco_Pagina_11.xlsx', index=False)

In [140]:
dfs_por_pagina["pagina_11"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Modalidade de Venda  9 non-null      object
 1   None                 9 non-null      object
 2   Descrição            9 non-null      object
dtypes: object(3)
memory usage: 344.0+ bytes


In [141]:
# Substituir colunas sem nome (None) por 'Legenda'
dfs_por_pagina["pagina_11"].columns = dfs_por_pagina["pagina_11"].columns.fillna('Legenda')

dfs_por_pagina["pagina_11"]['Legenda'] = dfs_por_pagina["pagina_11"]['Legenda'].str.replace('\n', ' ', regex=False)
dfs_por_pagina["pagina_11"]['Descrição'] = dfs_por_pagina["pagina_11"]['Descrição'].str.replace('\n', ' ', regex=False)

dfs_por_pagina["pagina_11"].head()

,Modalidade de Venda,Legenda,Descrição
0,EXA,Ex-Ponto “A”,"Entrega por duto a serviço da COMPRADORA, inte..."
1,LPA,Livre para o Armazém,"Entrega por duto, ou trecho de duto, a serviço..."
2,LCT,Livre no Compartimento de Carga do Veículo Rec...,Entrega diretamente no Veiculo Recebedor estac...
3,LPC,Livre para Carregamento,Entrega diretamente na Estação de Carregamento...
4,ETM,Entrega no Tanque de Destino Marítimo,Entrega no terminal de destino a partir de nav...


In [142]:
dfs_por_pagina["pagina_11"].to_excel('C:/Users/Samsung/Documents/GitHub-Datas/Case_Petrobahia/Tabela_de_Preco_Pagina_11.xlsx', index=False)

In [163]:
import re

# Função para corrigir palavras unidas
def corrigir_palavras_unidas(texto):
    # Garantir que é uma string
    texto = str(texto)
    
    # Substituir quebras de linha por espaço
    texto = texto.replace('\n', ' ')
    
    # Corrigir palavras unidas por maiúsculas
    texto_corrigido = re.sub(r'([a-z])([A-Z])', r'\1 \2', texto)
    
    # Corrigir outras palavras coladas (letras e números)
    texto_corrigido = re.sub(r'([a-zA-Z])([0-9])', r'\1 \2', texto_corrigido)
    texto_corrigido = re.sub(r'([0-9])([a-zA-Z])', r'\1 \2', texto_corrigido)
    
    return texto_corrigido

# Testar diretamente na célula com índice 6
dfs_por_pagina["pagina_11"]['Descrição'].iloc[6] = corrigir_palavras_unidas(dfs_por_pagina["pagina_11"]['Descrição'].iloc[6])

# Verificar o resultado
print(dfs_por_pagina["pagina_11"]['Descrição'].iloc[6])


Entrega diretamente na Estação de Descarregamento a serviço da COMPRADORA a partir da descargado Veículode Entrega,oudiretamentenotanquerecebedor,quandoaentregaseder por duto a serviço da PETROBRAS.


In [164]:
import nltk
nltk.download('punkt')  # Baixar os recursos necessários

from nltk.tokenize import word_tokenize

# Função para corrigir palavras unidas usando tokenização
def corrigir_palavras_unidas_nltk(texto):
    # Garantir que é uma string
    texto = str(texto)
    
    # Substituir quebras de linha por espaço
    texto = texto.replace('\n', ' ')
    
    # Tokenizar o texto em palavras
    palavras = word_tokenize(texto)
    
    # Reconstruir o texto com as palavras separadas corretamente
    texto_corrigido = ' '.join(palavras)
    
    return texto_corrigido

# Aplicar na célula com índice 6
dfs_por_pagina["pagina_11"]['Descrição'].iloc[6] = corrigir_palavras_unidas_nltk(dfs_por_pagina["pagina_11"]['Descrição'].iloc[6])

# Verificar o resultado
print(dfs_por_pagina["pagina_11"]['Descrição'].iloc[6])


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Samsung\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Entrega diretamente na Estação de Descarregamento a serviço da COMPRADORA a partir da descargado Veículode Entrega , oudiretamentenotanquerecebedor , quandoaentregaseder por duto a serviço da PETROBRAS .


In [169]:
!pip install spacy

  Using cached spacy-3.8.2.tar.gz (1.3 MB)
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'error'


  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [91 lines of output]
      Ignoring numpy: markers 'python_version >= "3.9"' don't match your environment
        Using cached setuptools-75.3.2-py3-none-any.whl (1.3 MB)
        Using cached Cython-0.29.37-py2.py3-none-any.whl (989 kB)
        Using cached cymem-2.0.11.tar.gz (10 kB)
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): finished with status 'done'
        Using cached preshed-3.0.9-cp38-cp38-win_amd64.whl (122 kB)
        Using cached murmurhash-1.0.12.tar.gz (13 kB)
        Installing build dependencies: started
        Installing build dependencies: f

In [170]:
!python -m spacy download pt_core_news_sm

c:\ProgramData\anaconda3\python.exe: No module named spacy


In [173]:
import sys
print(sys.executable)


c:\ProgramData\anaconda3\python.exe


In [174]:
import spacy
nlp = spacy.load('pt_core_news_sm')
print("Spacy instalado corretamente!")

ModuleNotFoundError: No module named 'spacy'

In [171]:
import spacy

# Carregar o modelo de linguagem em português do spacy
nlp = spacy.load('pt_core_news_sm')  # Você pode precisar instalar esse modelo

# Função para corrigir palavras unidas usando spacy
def corrigir_palavras_unidas_spacy(texto):
    # Garantir que é uma string
    texto = str(texto)
    
    # Substituir quebras de linha por espaço
    texto = texto.replace('\n', ' ')
    
    # Processar o texto com o modelo spacy
    doc = nlp(texto)
    
    # Reconstruir o texto corrigido (tokens separados por espaço)
    texto_corrigido = ' '.join([token.text for token in doc])
    
    return texto_corrigido

# Aplicar na célula com índice 6
dfs_por_pagina["pagina_11"]['Descrição'].iloc[6] = corrigir_palavras_unidas_spacy(dfs_por_pagina["pagina_11"]['Descrição'].iloc[6])

# Verificar o resultado
print(dfs_por_pagina["pagina_11"]['Descrição'].iloc[6])


ModuleNotFoundError: No module named 'spacy'

In [175]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download pt_core_news_sm

  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [91 lines of output]
      Ignoring numpy: markers 'python_version >= "3.9"' don't match your environment
        Using cached setuptools-75.3.2-py3-none-any.whl (1.3 MB)
        Using cached Cython-0.29.37-py2.py3-none-any.whl (989 kB)
        Using cached cymem-2.0.11.tar.gz (10 kB)
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): finished with status 'done'
        Using cached preshed-3.0.9-cp38-cp38-win_amd64.whl (122 kB)
        Using cached murmurhash-1.0.12.tar.gz (13 kB)
        Installing build dependencies: started
        Installing build dependencies: f


  Using cached spacy-3.8.2.tar.gz (1.3 MB)
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'error'


c:\ProgramData\anaconda3\python.exe: No module named spacy


In [176]:
import spacy
nlp = spacy.load("pt_core_news_sm")
print("Spacy instalado corretamente!")

ModuleNotFoundError: No module named 'spacy'

In [179]:
!pip install "spacy<3.5"
!python -m spacy download pt_core_news_sm


  error: subprocess-exited-with-error


     ---------------------------------------- 12.2/12.2 MB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 2.3/2.3 MB 6.7 MB/s eta 0:00:00
     ---------------------------------------- 182.4/182.4 kB ? eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 8.0 MB/s eta 0:00:00
  Using cached cymem-2.0.11.tar.gz (10 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 57.0/57.0 kB ? eta 0:00:00
     ---------------------------------------- 47.3/47.3 kB 2.5 MB/s eta 0:00:00
  Using cached preshed-3.0.9-cp38-cp38-win_amd64.whl (122 kB)
  Using cached murmurhash-1.0.12.tar.gz (13 kB)
  Installing build dependencies: started
  Installin


  
  × Building wheel for cymem (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-38\cymem
      copying cymem\about.py -> build\lib.win-amd64-cpython-38\cymem
      copying cymem\__init__.py -> build\lib.win-amd64-cpython-38\cymem
      creating build\lib.win-amd64-cpython-38\cymem\tests
      copying cymem\tests\test_import.py -> build\lib.win-amd64-cpython-38\cymem\tests
      copying cymem\tests\__init__.py -> build\lib.win-amd64-cpython-38\cymem\tests
      copying cymem\cymem.pyx -> build\lib.win-amd64-cpython-38\cymem
      copying cymem\cymem.pxd -> build\lib.win-amd64-cpython-38\cymem
      copying cymem\__init__.pxd -> build\lib.win-amd64-cpython-38\cymem
      running build_ext
      building 'cymem.cymem' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://v

In [181]:
import spacy
nlp = spacy.load("pt_core_news_sm")
print("Spacy instalado corretamente!")


ModuleNotFoundError: No module named 'spacy'